In [1]:
!pip install torch datasets numpy matplotlib seaborn
!pip uninstall -y datasets fsspec huggingface_hub transformers tokenizers
!rm -rf ~/.cache/huggingface/datasets
!pip install datasets==2.14.7 fsspec==2023.10.0 huggingface_hub==0.17.3 transformers==4.35.2 tokenizers==0.15.0
!pip install torch-geometric

Found existing installation: datasets 2.14.7
Uninstalling datasets-2.14.7:
  Successfully uninstalled datasets-2.14.7
Found existing installation: fsspec 2023.10.0
Uninstalling fsspec-2023.10.0:
  Successfully uninstalled fsspec-2023.10.0
Found existing installation: huggingface-hub 0.17.3
Uninstalling huggingface-hub-0.17.3:
  Successfully uninstalled huggingface-hub-0.17.3
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
Found existing installation: tokenizers 0.15.0
Uninstalling tokenizers-0.15.0:
  Successfully uninstalled tokenizers-0.15.0
  Using cached datasets-2.14.7-py3-none-any.whl.metadata (19 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached huggingface_hub-0.17.3-py3-none-any.whl.metadata (13 kB)
  Using cached transformers-4.35.2-py3-none-any.whl.metadata (123 kB)
  Using cached tokenizers-0.15.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.me

In [8]:
import torch
print(torch.__version__)


2.7.0+cu126


In [3]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.7.0+cu126.html

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html


<h1>Optimized Test Run</h1>


In [ ]:
!python run_gnn_moe.py \
  --coupler_type HGNN \
  --static_hyperedge_strategy all_pairs \
  --hgnn_learnable_edge_weights \
  --embed_dim 512 \
  --num_layers 8 \
  --batch_size 16 \
  --learning_rate 0.0005 \
  --gnn_layers 3 \
  --num_experts 2 \
  --epochs 10 \
  --eval_every 250 \
  --run_name "hgnn_vs_gnn_optimal_showdown" \
  --dataset_config_name wikitext-2-v1


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
Adjusting num_heads from 8 to 4 based on embed_dim 256
Adjusting num_heads from 8 to 4 based on embed_dim 256
Overriding config.embed_dim with CLI arg: 512
Overriding config.num_layers with CLI arg: 8
Overriding config.num_experts with CLI arg: 2
Overriding config.gnn_layers with CLI arg: 3
Overriding config.coupler_type with CLI arg: HGNN
Overriding config.batch_size with CLI arg: 16
Overriding config.epochs with CLI arg: 10
Overriding config.eval_every with CLI arg: 250
===== GNN-MoE Hyperparameter Script Execution Started =====
Run Name: hgnn_vs_gnn_optimal_showdown
Effective Config: GNNMoEConfig(vocab_size=50257, max_seq_length=128, embed_dim=512, num_layers=8, num_heads=4, dropout_rate=0.1, num_experts=2, gnn_layers=3, batch_size=16, 

<h1>Ghost Ops</h1>

## Ghost Expert Architecture Training


In [ ]:
# Ghost Expert Architecture Training
!python -m ghost.run_gnn_moe \
  --embed_dim 256 \
  --num_layers 4 \
  --num_experts 4 \
  --num_ghost_experts 4 \
  --ghost_activation_threshold 0.75 \
  --ghost_learning_rate 1e-4 \
  --epochs 3 \
  --max_batches_per_epoch 100 \
  --eval_every 50 \
  --run_name "ghost_notebook_demo" \
  --dataset_config_name wikitext-2-v1


In [ ]:
# Test different ghost configurations
!python ghost/tests/run_ghost_sweep.py


In [ ]:
# Test different architecture configurations
!python ghost/tests/run_architecture_sweep.py


In [ ]:
import glob
import os

# Find latest sweep
sweep_dirs = glob.glob('ghost/tests/sweeps/*')
if sweep_dirs:
    latest_sweep = max(sweep_dirs, key=os.path.getctime)
    print(f"Analyzing: {latest_sweep}")
    !python ghost/tests/enhanced_analysis.py {latest_sweep}
else:
    print("No sweeps found")


### Load and Visualize

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

# Load latest training log
log_files = glob.glob('ghost/tests/sweeps/*/*/training_log.json')
if log_files:
    latest_log = max(log_files, key=os.path.getctime)
    with open(latest_log, 'r') as f:
        training_data = json.load(f)
    
    df = pd.DataFrame(training_data)
    
    # Plot ghost activations over time
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Ghost activations
    ghost_activations = pd.DataFrame(df['ghost_activations'].tolist(), index=df['step'])
    ghost_activations.plot(ax=ax1, title='Ghost Expert Activation Levels')
    ax1.set_ylabel('Activation Level')
    
    # Saturation vs Orthogonality
    ax2.plot(df['step'], df['saturation_level'], label='Saturation', color='red')
    ax2_twin = ax2.twinx()
    ax2_twin.plot(df['step'], df['orthogonality_score'], label='Orthogonality', color='blue')
    ax2.set_title('Expert Dynamics')
    ax2.set_xlabel('Training Step')
    ax2.legend(loc='upper left')
    ax2_twin.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Final ghost activations: {df.iloc[-1]['ghost_activations']}")
    print(f"Final saturation: {df.iloc[-1]['saturation_level']:.3f}")
else:
    print("No training logs found")
